# Entrada y salida de imágenes

## Representación
Perceptualmente, una imagen digital consta de una secuencia de intensidades (píxeles) agrupadas en una matriz de dimension $H \times W$. Inicialmente, cada valor de intensidad presente en la matriz puede encontrarse en el rango $[0, 255]$ o en el rango $[0, 1]$ (Normalizado). 

Ahora bien las imágenes pueden ser representadas a color o en escala de grises. En este caso, la variación radica en el número de matrices que representan a una imagen. Si una imagen presenta tres matrices (RGB inicialmente), entonces es posible afirmar que la imagen descrita contiene información cromática. Por el contrario, si una imagen se encuentra descrita por una única matriz, ésta se encuentra en escala de grises.

![alt text](./Assets/rgb_image.png "Imágen a color")

**Nota:** Si una imagen cuenta con una capa de transparencia, esta se encontrará descrita por 4 matrices. Sin embargo, esta información puede resultar de utilidad en aplicaciones de visualización.

A continuación se presentarán dos ejemplos en los cuales se presenta esta distinción, asímismo, se presentarán las funciones básicas de carga de imágenes en OpenCV. 

## Carga de imágenes
Debido a que las imágenes se encuentran representadas a través de matrices, OpenCV opera sobre arreglos de Numpy, lo cual permite emplear las funciones de esta librería, así como ``scipy``, ``matplotlib``  y otras librerías que puedan realizar operaciones sobre matrices. 

En este caso, se presentan dos funciones diseñadas para la carga de imágenes en Python, la primera basada en ``matplotlib.image`` y la segunda, basada en OpenCV (``cv2``). La diferencia entre las dos radica principalmente en el orden de los canales RGB, mientras la función ``matplotlib.image.imread`` retorna las tres matrices en el orden $[R, G, B]$, la función ``cv2.imread`` retorna una imagen con canales $[B, G, R]$.

**Nota:** A lo largo del tutorial se empleará la función de Matplotlib.

Para visualizar las imágenes es posible usar la función ``matplotlib.pyplot.imshow``, como se presenta a continuación:

In [ ]:
# Importación de librerías básicas
import cv2 # OpenCV
import numpy as np # Numpy
import scipy.misc as smc
%matplotlib inline
import matplotlib.pyplot as plt # matplotlib.pyplot
import matplotlib.image as mpimg # matplotlib.image

### Carga y visualización de imágenes RGB

In [ ]:
img = mpimg.imread('img/DSC_0280.JPG')
ax = plt.imshow(img)
print("Dimensiones: {0}".format(img.shape)) # Las dimensiones de la imagen

## Espacios de color
En ocasiones, el espacio de representación RGB es limitado para realizar ciertas operaciones sobre las imágenes, como por ejemplo, para hallar las regiones de la imágen que son de un color específico, o para encontrar el gradiente de la imagen, como se presentará en el siguiente módulo. En estos casos, otros espacios de color como la escala de grises o [HSV](https://en.wikipedia.org/wiki/HSL_and_HSV) pueden ser óptimos para estas aplicaciones. Para este fin, OpenCV dispone de la función ``cv2.cvtColor``, la cual permite transformar una imagen desde un espacio de representación a otro.

Las conversiones permitidas por la función se encuentran descritas a través de un conjunto de constantes ``cv2.COlOR_*``, presentes en el módulo ``cv2``. En el presente ejemplo, se emplearán las constantes ``cv2.COLOR_RGB2GRAY`` y ``cv2.COLOR_RGB2HSV``, sin embargo, el listado completo de dichas constantes se puede obtener con la siguiente instrucción: 

In [ ]:
flags = [i for i in dir(cv2) if i.startswith('COLOR_')]
print(flags)

A continuación se procede a cambiar el espacio de representación de la imagen cargada

In [ ]:
gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
ax = plt.imshow(gray, cmap='gray') # Nótese el uso del keyword cmap para visualizar la imagen en escala de grises
print('Dimensiones: {0}'.format(gray.shape))

In [ ]:
hsv_img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
ax = plt.imshow(hsv_img) # Notar que las regiones que comparten un color tienen valores de intensidad HSV similares

Debido a que la función ``plt.imshow`` espera una imagen representada en el espacio RGB, si se carga una imagen en escala de grises, hsv u otro espacio de color, es necesario cambiar de mapa de colores a través del argumento ``cmap``. La lista completa de mapas de colores disponibles en matplotlib se encuentra en: http://matplotlib.org/users/colormaps.html

## Operaciones sobre canales de color
Debido a que una imagen se comporta como una matriz de numpy, es posible realizar las operaciones habituales sobre las mismas, por ejemplo, acceder al valor de un índice en específico, realizar slicing (rebanado) y otras operaciones sobre una matriz. _e.g.,_

In [ ]:
# Recortar una imagen usando slicing
crop = img[0:3000, 100:2000, :] # Se realiza la operación sobre los tres canales
plt.imshow(crop)

In [ ]:
# Extraer los canales individuales de una imagen
r, g, b = np.rollaxis(img[:,:,...], axis=-1)
plt.imshow(b, cmap='gray')

In [ ]:
# Combinar tres canales de color para formar una imagen
full_img = np.dstack((r, g, b))
plt.imshow(full_img)

In [ ]:
# Hallar el máximo valor de intensidad en la imagen
np.max(img)

In [ ]:
# Calcular la raíz cuadrada de la imagen
plt.imshow(np.sqrt(img))

In [ ]:
# Cambiar los valores de una region
img[2500:4000, 0:2000, :] = img[2500:4000, 4000:6000, :]
plt.imshow(img)

## Operaciones Aritméticas entre imágenes
Además de las funciones incluidas en numpy, es posible emplear las funciones nativas presentes en OpenCV, estas permiten realizar operaciones de suma (``cv2.add``), suma ponderada (``cv2.addWeigthed``), así como operaciones binarias como el and (``cv2.bitwise_and``), or (``cv2.bitwise_or``) y not lógicos (``cv2.bitwise_not``). Es necesario notar que estas funciones son saturadas, esto implica que en caso de que una suma desborde el valor de 255, el resultado será equivalente a este valor. 

A continuación se presenta el uso de las función de suma ponderada, la cual es útil para sobreponer dos imágenes, las otras funciones serán empleadas más adelante en el tutorial.

### Suma ponderada de imágenes
Esta función calcula la superposición de dos imágenes, cada una con un peso de $\alpha$ y $\beta$, respectivamente. Adicionalmente, la función admite la inclusión de un parámetro de equalización $\gamma$, el cual es añadido al resultado final. En conclusión, dadas dos imágenes $\mathcal{Im1}$ e $\mathcal{Im2}$, la función ``cv2.addWeighted`` calcula:
$$
\mathcal{dst} = \alpha \cdot \mathcal{Im1} + \beta \cdot \mathcal{Im2} + \gamma 
$$

Si se desea realizar una transición suave entre dos imágenes, es posible emplear un solo peso $\alpha$ para ambas imágenes, como se presenta a continuación:

$$
\mathcal{dst} = (1 - \alpha) \cdot \mathcal{Im1} + \alpha \cdot \mathcal{Im2} 
$$


**Nota:** Si $\alpha$ inicia en cero e incrementa lentamente hasta uno, es posible observar una transición suave entre las imágenes.

In [1]:
alpha = 0.5
beta = 1 - alpha
gamma = 0

img1 = mpimg.imread('img/blend/12003.jpg')
img2 = mpimg.imread('img/blend/22090.jpg')

plt.imshow(img1)
plt.figure()
plt.imshow(img2)

NameError: name 'mpimg' is not defined

In [ ]:
dst = cv2.addWeighted(img1, alpha, img2, beta, gamma) 
plt.imshow(dst)

## Guardar imágenes

Al igual que la función ``imread``, presente tanto en el módulo ``cv2``, aśi como en el módulo ``matplotlib.image`` se empleaba para cargar una imagen, la función ``imsave`` presente en ambos módulos permite guardar una imagen en un archivo binario. Una vez más, la diferencia entre las funciones en cuestión radica en el orden de los canales de la imagen, mientras la función de OpenCV espera una imagen BGR, la función de matplotlib espera una imagen en orden RGB.

Por ejemplo, es posible guardar el resultado de la operación de suma ponderada presentado previamente:

In [ ]:
mpimg.imsave('img/dst.jpg', dst)

Es posible verificar que la imagen se encuentra guardada:

In [ ]:
import os.path as osp

print("La imagen existe: {0}".format(osp.isfile('img/dst.jpg')))

img = mpimg.imread('img/dst.jpg')
plt.imshow(img)

## Visualización de imágenes en pygame

A continuación, se introduce al uso de [Pygame](https://www.pygame.org/news), una librería orientada al prototipado y desarrollo de videojuegos sencillos en python. Esta librería se encuentra implementada encima del [Simple DirectMedia Layer](https://en.wikipedia.org/wiki/Simple_DirectMedia_Layer), una librería gráfica que pretende abstraer el funcionamiento del hardware gráfico y de sonido, independiente del Sistema Operativo empleado. 

En este caso, se empleará pygame para visualizar secuencias de imágenes (Video) en tiempo real, sin embargo, más adelante en el tutorial se realizará la demostración de un juego basado en el procesamiento de imágenes realizado en OpenCV.

A continuación se presentará la implementación de un sencillo _slideshow_ de imágenes a partir de archivos presentes en la carpeta ``img/slideshow``

**Nota:** Los ejemplos presentados a continuación deben ser ejecutados directamente en una consola de Python, para este fin, es posible editar en Spyder los archivos situados bajo la carpeta ``pygame_basic`` del presente tutorial.

### 1. Librerías e inicialización
En primer lugar, es necesario importar las librerías necesarias para la implementación:

```python
import cv2  # OpenCV
import glob  # Glob: Librería para listar archivos
import time  # Control de tiempo
import pygame  # pygame
import numpy as np  # numpy
from pygame.locals import *  # Todas las teclas y eventos de pygame
import matplotlib.image as mpimg  # Rutinas de entrada/salida de imágenes
```

### 2. Rutina principal

A continuación se procede a dibujar las imagenes en la ventana de pygame, cada imagen tarda 1 segundo en pantalla
```python
def main():
    """
    Rutina principal
    """
    fps = 1  # 1 fotogramas por segundo
    finish = False  # El programa no ha finalizado
    pygame.init()  # Se inicializa pygame
    
    files = glob.glob('../img/slideshow/*.jpg')  # Se recuperan todas las imágenes JPEG
    cur_img = 0  # Índice de la imagen actual
    img = mpimg.imread(files[0])  # Se carga la primera imagen
    
    size = img.shape  # Se obtienen las dimensiones de la imagen
    screen = pygame.display.set_mode((size))  # Se define el tamaño de la ventana
    pygame.display.set_caption('Slideshow')  # Se define el título de la ventana
    
    clock = pygame.time.Clock()  # Control de fotogramas
    while not finish:  # Mientras no haya finalizado
        surf = pygame.surfarray.make_surface(img)  # Se carga la imagen a una superficie
        screen.blit(surf, (0, 0))  # Se muestra la superficie en la ventana
        pygame.display.update()  # Se muestra la imagen en pantalla
        clock.tick(FPS)  # Cuenta de fotogramas
        
        cur_img = (cur_img + 1) % len(files)  # Se aumenta el índice
        img = mpimg.imread(files[cur_img])  # Se carga la siguiente imagen
        
        for event in pygame.event.get(): 
            if event.type == pygame.QUIT:  # Si el usuario cierra la ventana
                finish = True
    
```

### 3. Punto de entrada a la aplicación

Esto permite ejecutar la función main si se invoca el archivo en consola con el comando ``python slideshow.py``
```python
if __name__ == '__main__':
    main()
```

Tras ejecutar este programa, este debería ser el resultado:

![alt text](./Assets/slideshow.gif "Slideshow de imágenes usando pygame")

## Ejercicios

## Ejercicio 1
Cargar una imagen a color, cambiar el espacio de representación a escala de grises y guardar el resultado.

## Ejercicio 2
Modificar el ejemplo de pygame para hacer una transición suave entre las imágenes.

![alt text](./Assets/smooth.gif "Transición suave de imágenes")